In [1]:
import pandas as pd
import re

### Sample of Reported Titles

Provide job titles reported during ONET data collection
This file contains job titles frequently reported by incumbents and occupational experts on data collection surveys.

### Alternate Titles

Provide alternate occupational titles for ONET-SOC occupations
This file contains alternate, or "lay", occupational titles for the ONET-SOC classification system. The file was developed to improve keyword searches in several Department of Labor internet applications (i.e., Career InfoNet, ONET OnLine, and O*NET Code Connector). The file contains occupational titles from existing occupational classification systems, as well as from other diverse sources. When a title contains acronyms, abbreviations, or jargon, the “Short Title” column contains the brief version of the full title. 

In [2]:
df_sample_titles = pd.read_excel("Sample of Reported Titles.xlsx")
df_sample_titles.rename(columns={'O*NET-SOC Code': 'soc', 'Reported Job Title': 'original_title'}, inplace=True)

df_sample_titles = df_sample_titles[['soc','original_title']]
df_sample_titles.head()

# https://www.onetcenter.org/dl_files/database/db_21_1_dictionary.pdf
# page 52

,soc,original_title
0,11-1011.00,Chief Diversity Officer (CDO)
1,11-1011.00,Chief Executive Officer (CEO)
2,11-1011.00,Chief Financial Officer (CFO)
3,11-1011.00,Chief Nursing Officer
4,11-1011.00,Chief Operating Officer (COO)


In [3]:
df_alternate_titles = pd.read_excel('Alternate Titles.xlsx')
df_alternate_titles.rename(columns={'O*NET-SOC Code': 'soc', 'Alternate Title': 'original_title'}, inplace=True)

df_alternate_titles = df_alternate_titles[['soc','original_title']]
df_alternate_titles.head()

# https://www.onetcenter.org/dl_files/database/db_21_1_dictionary.pdf
# page 50

,soc,original_title
0,11-1011.00,Aeronautics Commission Director
1,11-1011.00,Agricultural Services Director
2,11-1011.00,Alcohol and Drug Abuse Assistance Program Admi...
3,11-1011.00,Arts and Humanities Council Director
4,11-1011.00,Bakery Manager


In [4]:
df = df_sample_titles.append(df_alternate_titles, ignore_index=True)
df = df.drop_duplicates()

In [5]:
def clean_title(title):
    remove_word = ['of','the','and','or']
    cleaned = title.lower()
    cleaned = cleaned.replace('.','')
    cleaned = re.sub('[^a-z0-9]',' ',cleaned)
    cleaned_tokens = [w for w in re.split(' ',cleaned) if not w in remove_word]
    cleaned_tokens = sorted([w for w in cleaned_tokens if not w==''])
    return ' '.join(cleaned_tokens)

In [6]:
df['cleaned_title'] = df['original_title'].apply(lambda x: clean_title(x))
df.head(20)

,soc,original_title,cleaned_title
0,11-1011.00,Chief Diversity Officer (CDO),cdo chief diversity officer
1,11-1011.00,Chief Executive Officer (CEO),ceo chief executive officer
2,11-1011.00,Chief Financial Officer (CFO),cfo chief financial officer
3,11-1011.00,Chief Nursing Officer,chief nursing officer
4,11-1011.00,Chief Operating Officer (COO),chief coo officer operating
5,11-1011.00,Executive Director,director executive
6,11-1011.00,Executive Vice President (EVP),evp executive president vice
7,11-1011.00,Operations Vice President,operations president vice
8,11-1011.00,President,president
9,11-1011.00,Vice President,president vice


In [7]:
len(df)

59972

In [8]:
len(df.cleaned_title.unique())

45544

In [9]:
list_duplicating_titles = list()
list_cleaned_title = list()
list_cleaned_title2soc = list()
list_duplicates_with_soc = list()

for index, row in df.iterrows():
    original_title = str(row['original_title'])
    cleaned_title = str(row['cleaned_title'])
    soc = str(row['soc'])
    cleaned_title2soc = cleaned_title + '\t' + soc
    
    if cleaned_title in list_cleaned_title:
        if cleaned_title2soc in list_cleaned_title2soc:
            pass
        else:
            list_duplicating_titles.append(cleaned_title)
    else:
        list_cleaned_title.append(cleaned_title)
        list_cleaned_title2soc.append(cleaned_title2soc)

list_duplicating_titles = sorted(list(set(list_duplicating_titles)))   
n_duplicating_titles = len(list_duplicating_titles)
print('Total duplicating titles = ' + str(n_duplicating_titles) )
        
for index, row in df.iterrows():
    original_title = row['original_title']
    cleaned_title = row['cleaned_title']
    onet = str(row['soc'])
    
    if cleaned_title in list_duplicating_titles:
        loc = list_duplicating_titles.index(cleaned_title)
        output = [str(loc), cleaned_title, original_title, onet]
        list_duplicates_with_soc.append( '\t'.join(output) )

list_duplicates_with_soc = sorted(list_duplicates_with_soc)

duplicating_titles_file = open('duplicating_titles_2016.txt','w')
duplicating_titles_file.write( '\n'.join(list_duplicates_with_soc) )
duplicating_titles_file.close()

list_cleaned_title_file = open('cleaned_title_2016.txt','w')
list_cleaned_title_file.write( '\n'.join(list_cleaned_title) )
list_cleaned_title_file.close()

Total duplicating titles = 8268


In [10]:
list_duplicates_2digit = list()
list_duplicates_2digit_with_soc = list()
list_title2soc_2digit = list()
list_cleaned_title = list()

list_all_duplicates = open('duplicating_titles_2016.txt')

for line in list_all_duplicates:
    line_clean = re.sub('\n','',line)
    if not line == '':
        split = re.split('\t',line_clean)
        assert( len(split) == 4 )
        cleaned_title = split[1]
        original_title = split[2]
        soc2digit = str(re.sub('\D','',split[3]))[:2]
        title2soc_2digit = cleaned_title + '\t' + str(soc2digit)
        
        if cleaned_title in list_cleaned_title:
            if title2soc_2digit in list_title2soc_2digit:
                pass
            else:
                list_duplicates_2digit.append(cleaned_title)
        else:
            list_cleaned_title.append(cleaned_title)
            list_title2soc_2digit.append(title2soc_2digit)

list_duplicates_2digit = sorted(list(set(list_duplicates_2digit)))   
n_list_duplicates_2digit = len(list_duplicates_2digit)
print('Total duplicating titles (2 digits soc) = ' + str(n_list_duplicates_2digit) )

list_all_duplicates = open('duplicating_titles_2016.txt')

for line in list_all_duplicates:
    line_clean = re.sub('\n','',line)
    if not line == '':
        split = re.split('\t',line_clean)
        assert( len(split) == 4 )
        cleaned_title = split[1]
        original_title = split[2]
        soc = str(split[3])
        title2soc = cleaned_title + '\t'+ original_title + '\t' + soc
        
        if cleaned_title in list_duplicates_2digit: 
            list_duplicates_2digit_with_soc.append(title2soc)
            
duplicating_title_2digit = open('duplicating_titles_2digit_2016.txt','w')
duplicating_title_2digit.write( '\n'.join(list_duplicates_2digit_with_soc) )
duplicating_title_2digit.close()

Total duplicating titles (2 digits soc) = 3577
